<a href="https://colab.research.google.com/github/deepika-3/Deepa/blob/main/con2d_to_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision.models import densenet121
from torchvision.models._utils import IntermediateLayerGetter


def conv3d_from_2d(conv2d):
    return nn.Conv3d(
        in_channels=conv2d.in_channels,
        out_channels=conv2d.out_channels,
        kernel_size=conv2d.kernel_size,
        stride=conv2d.stride,
        padding=conv2d.padding,
        bias=conv2d.bias is not None
    )


def batchnorm3d_from_2d(bn2d):
    return nn.BatchNorm3d(
        num_features=bn2d.num_features,
        eps=bn2d.eps,
        momentum=bn2d.momentum,
        affine=bn2d.affine,
        track_running_stats=bn2d.track_running_stats
    )


def convert_conv2d_to_conv3d(module):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            setattr(module, name, conv3d_from_2d(child))
        elif isinstance(child, nn.BatchNorm2d):
            setattr(module, name, batchnorm3d_from_2d(child))
        elif isinstance(child, nn.AdaptiveAvgPool2d):
            setattr(module, name, nn.AdaptiveAvgPool3d((1, 1, 1)))
        elif isinstance(child, nn.MaxPool2d):
            setattr(module, name, nn.MaxPool3d(kernel_size=3, stride=2, padding=1))
        else:
            convert_conv2d_to_conv3d(child)
    return module


# Load pretrained 2D model
model_2d = densenet121(pretrained=True)

# Convert to 3D
model_3d = convert_conv2d_to_conv3d(model_2d)

# Replace the classifier (optional: based on your task)
model_3d.classifier = nn.Linear(in_features=1024, out_features=2)  # For binary classification

print(model_3d)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 156MB/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv3d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv3d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm3d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# --- Step 1: Inflate Conv2D to Conv3D ---
def inflate_conv2d(conv2d, depth=3):
    conv3d = nn.Conv3d(
        in_channels=conv2d.in_channels,
        out_channels=conv2d.out_channels,
        kernel_size=(depth, *conv2d.kernel_size),
        stride=(1, *conv2d.stride),
        padding=(depth // 2, *conv2d.padding),
        bias=conv2d.bias is not None
    )
    with torch.no_grad():
        weight2d = conv2d.weight.data  # (out_ch, in_ch, H, W)
        weight3d = weight2d.unsqueeze(2).repeat(1, 1, depth, 1, 1) / depth
        conv3d.weight.copy_(weight3d)
        if conv2d.bias is not None:
            conv3d.bias.copy_(conv2d.bias.data)
    return conv3d

# --- Step 2: Replace 2D layers with 3D versions recursively ---
def convert_layer_to_3d(module):
    if isinstance(module, nn.Conv2d):
        return inflate_conv2d(module, depth=3)
    elif isinstance(module, nn.BatchNorm2d):
        new_bn = nn.BatchNorm3d(module.num_features)
        new_bn.load_state_dict(module.state_dict())
        return new_bn
    elif isinstance(module, nn.AdaptiveAvgPool2d):
        return nn.AdaptiveAvgPool3d((1, 1, 1))
    elif isinstance(module, nn.MaxPool2d):
        return nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
    else:
        return module

def apply_3d_conversion(model):
    for name, child in model.named_children():
        new_child = convert_layer_to_3d(child)
        setattr(model, name, new_child)
        apply_3d_conversion(new_child)
    return model

# --- Step 3: Load pretrained DenseNet121 and convert ---
def get_3d_densenet121(num_classes=2):
    model_2d = models.densenet121(pretrained=True)
    model_3d = apply_3d_conversion(model_2d)
    model_3d.classifier = nn.Linear(1024, num_classes)  # Adjust to your output classes
    return model_3d

# --- Step 4: Optional - test with a random 3D input ---
if __name__ == '__main__':
    model_3d = get_3d_densenet121(num_classes=2)
    model_3d.eval()
    x = torch.randn(1, 3, 3, 224, 224)  # (Batch, Channels, Depth, Height, Width)
    with torch.no_grad():
        output = model_3d(x)
        print("✅ Output shape:", output.shape)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:01<00:00, 22.3MB/s]


RuntimeError: Expected 3D or 4D (batch mode) tensor with optional 0 dim batch size for input, but got:[1, 128, 3, 56, 56]